In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!nvidia-smi

Tue Apr 16 04:32:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install diffusers
!pip install transformers scipy ftfy accelerate ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 775.4 kB/s eta 0:00:00a 0:00:01


In [3]:
!pip install torch torchvision torchaudio
!pip install bitsandbytes
!pip install peft
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 1.4 MB/s eta 0:00:00a 0:00:01m


In [4]:
import numpy as np
w_list = np.array([2,3,4,7])

import random
x_list = []
for _ in range(10):
    x_sample = np.array([random.randint(1,100) for _ in range(len(w_list))])
    x_list.append(x_sample)

y_list = []
for x_sample in x_list:
    y_temp = x_sample@w_list
    y_list.append(y_temp)

print("x_list:",x_list)
print("y_list:",y_list)

x_list: [array([94, 97, 12, 58]), array([86, 69, 22, 37]), array([50, 17, 45, 72]), array([18, 38, 26, 63]), array([34, 42, 10, 68]), array([83, 77, 30, 92]), array([69, 81, 41,  9]), array([40, 32, 15, 88]), array([60,  3, 39, 77]), array([98, 43, 12,  3])]
y_list: [933, 726, 835, 695, 710, 1161, 608, 852, 824, 394]


In [5]:
import torch 
import torch.nn as nn

class MyLinear(nn.Module):
    def __init__(self):
        super().__init__()
        self.w = nn.Parameter(torch.randn(4))

    def forward(self, x:torch.Tensor):
        return self.w @ x

In [6]:
model = MyLinear()

In [7]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.00001)

In [8]:
x_input = torch.tensor(x_list, dtype=torch.float32)
y_output = torch.tensor(y_list, dtype=torch.float32)

/tmp/ipykernel_34/198170575.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:261.)
  x_input = torch.tensor(x_list, dtype=torch.float32)


In [9]:
# start train model
num_epochs = 100
for epoch in range(num_epochs):
    for i, x in enumerate(x_input):
        # forward
        y_pred = model(x)

        # calculate loss
        loss = loss_fn(y_pred,y_output[i])

        # zero out the cached parameter.
        optimizer.zero_grad()

        # backward
        loss.backward()

        # update paramters
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

print("train done") 

Epoch [10/100], Loss: 3520.0327
Epoch [20/100], Loss: 792.9136
Epoch [30/100], Loss: 351.4698
Epoch [40/100], Loss: 165.5834
Epoch [50/100], Loss: 78.4869
Epoch [60/100], Loss: 37.2611
Epoch [70/100], Loss: 17.7042
Epoch [80/100], Loss: 8.4165
Epoch [90/100], Loss: 4.0024
Epoch [100/100], Loss: 1.9037
train done


In [10]:
model.w

Parameter containing:
tensor([2.0250, 2.9863, 3.9298, 7.0135], requires_grad=True)

In [11]:
from accelerate import utils
utils.write_basic_config()

PosixPath('/root/.cache/huggingface/accelerate/default_config.yaml')

In [15]:
from accelerate import utils
utils.write_basic_config()

from accelerate import Accelerator
accelerator = Accelerator()
device = accelerator.device

x_input.to(device)
y_output.to(device)
model.to(device)

model, optimizer = accelerator.prepare(
    model, optimizer
)

num_epochs = 100
for epoch in range(num_epochs):
    for i, x in enumerate(x_input):
        # forward
        y_pred = model(x)

        # calculate loss
        loss = loss_fn(y_pred,y_output[i])

        # zero out the cached parameter.
        optimizer.zero_grad()

        # backward
        #loss.backward()
        accelerator.backward(loss)

        # update paramters
        optimizer.step()

    if (epoch+1) % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

print("train done") 

MyLinear()

In [16]:
model = accelerator.unwrap_model(model)
model.w

Parameter containing:
tensor([2.0250, 2.9863, 3.9298, 7.0135], device='cuda:0', requires_grad=True)

In [19]:
import numpy as np
w_list = np.array([2,3,4,7])

import random
x_list = []
for _ in range(10):
    x_sample = np.array([random.randint(1,100) for _ in range(len(w_list))])
    x_list.append(x_sample)

y_list = []
for x_sample in x_list:
    y_temp = x_sample@w_list
    y_list.append(y_temp)
train_obj = {
    'w_list':w_list.tolist()
    , 'input':x_list
    , 'output':y_list
}

import pickle
with open('train_data.pkl','wb') as f:
    pickle.dump(train_obj,f)

In [20]:
import torch
import torch.nn as nn
from accelerate import utils
from accelerate import Accelerator

# start a accelerate instance
utils.write_basic_config()
accelerator = Accelerator()
device = accelerator.device

def main():
    # define the model
    class MyLinear(nn.Module):
        def __init__(self):
            super().__init__()
            self.w = nn.Parameter(torch.randn(len(w_list)))
        
        def forward(self, x:torch.Tensor):
            return self.w @ x

    # load training data
    import pickle
    with open("train_data.pkl",'rb') as f:
        loaded_object = pickle.load(f)
    w_list = loaded_object['w_list']
    x_list = loaded_object['input']
    y_list = loaded_object['output']

    # convert data to torch tensor
    x_input     = torch.tensor(x_list, dtype=torch.float32).to(device)
    y_output    = torch.tensor(y_list, dtype=torch.float32).to(device)

    # initialize model, loss function, and optimizer
    model       = MyLinear().to(device)
    loss_fn     = nn.MSELoss()
    optimizer   = torch.optim.SGD(model.parameters(), lr = 0.00001)

    # wrap model and optimizer using accelerate
    model, optimizer = accelerator.prepare(
        model, optimizer
    )

    num_epochs = 100
    for epoch in range(num_epochs):
        for i, x in enumerate(x_input):
            # forward
            y_pred = model(x)

            # calculate loss
            loss = loss_fn(y_pred,y_output[i])

            # zero out the cached parameter.
            optimizer.zero_grad()

            # backward
            #loss.backward()
            accelerator.backward(loss)

            # update paramters
            optimizer.step()

        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
    
    # take a look at the model weights after trainning
    model = accelerator.unwrap_model(model)
    print(model.w)

if __name__ == "__main__":
    main()

Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.
Epoch [10/100], Loss: 124.7672
Epoch [20/100], Loss: 158.8025
Epoch [30/100], Loss: 91.4274
Epoch [40/100], Loss: 39.8153
Epoch [50/100], Loss: 15.3667
Epoch [60/100], Loss: 5.5932
Epoch [70/100], Loss: 1.9755
Epoch [80/100], Loss: 0.6868
Epoch [90/100], Loss: 0.2368
Epoch [100/100], Loss: 0.0812
Parameter containing:
tensor([2.0098, 3.0060, 3.9959, 6.9913], device='cuda:0', requires_grad=True)


In [21]:
# import packages
import torch
from accelerate import utils
from accelerate import Accelerator
from diffusers import DDPMScheduler,StableDiffusionPipeline
from peft import LoraConfig
from peft.utils import get_peft_model_state_dict
from datasets import load_dataset
from torchvision import transforms
import math
from diffusers.optimization import get_scheduler
from tqdm.auto import tqdm
import torch.nn.functional as F
from diffusers.utils import convert_state_dict_to_diffusers

from datetime import datetime
formatted_date = datetime.now().strftime(r'%Y%m%d-%H%M%S')   

# train code 
def main():
    utils.write_basic_config()

    # hyperparameters
    output_dir                      = "."
    pretrained_model_name_or_path   = "runwayml/stable-diffusion-v1-5"
    lora_rank                       = 4
    lora_alpha                      = 4
    learning_rate                   = 1e-4
    adam_beta1, adam_beta2          = 0.9, 0.999
    adam_weight_decay               = 1e-2
    adam_epsilon                    = 1e-08
    dataset_name                    = None                  #"lambdalabs/pokemon-blip-captions"
    train_data_dir                  = "./train_data"
    top_rows                        = 4
    output_dir                      = "output_dir"
    resolution                      = 768
    center_crop                     = True
    random_flip                     = True
    train_batch_size                = 4
    gradient_accumulation_steps     = 1
    num_train_epochs                = 200
    # The scheduler type to use. Choose between ["linear", "cosine", "cosine_with_restarts", "polynomial","constant", "constant_with_warmup"]   
    lr_scheduler_name               = "constant" #"cosine"#
    max_grad_norm                   = 1.0
    diffusion_scheduler             = DDPMScheduler  

    accelerator = Accelerator(
        gradient_accumulation_steps = gradient_accumulation_steps
        , mixed_precision           = "fp16" 
    )
    device      = accelerator.device

    # Load scheduler, tokenizer and unet models.
    noise_scheduler = DDPMScheduler.from_pretrained(pretrained_model_name_or_path, subfolder="scheduler")
    weight_dtype = torch.float16
    pipe = StableDiffusionPipeline.from_pretrained(
        pretrained_model_name_or_path, 
        torch_dtype = weight_dtype
    ).to(device)
    tokenizer,text_encoder, vae, unet   = pipe.tokenizer, pipe.text_encoder, pipe.vae, pipe.unet

    # freeze parameters of models, we just want to train a LoRA only
    unet.requires_grad_(False)
    vae.requires_grad_(False)
    text_encoder.requires_grad_(False)

    # configure LoRA parameters use PEFT
    unet_lora_config = LoraConfig(
        r                     = lora_rank
        , lora_alpha          = lora_alpha
        , init_lora_weights   = "gaussian"
        , target_modules      = ["to_k", "to_q", "to_v", "to_out.0"]
    )

    # Add adapter and make sure the trainable params are in float32.
    unet.add_adapter(unet_lora_config)
    for param in unet.parameters():
        # only upcast trainable parameters (LoRA) into fp32
        if param.requires_grad:
            param.data = param.to(torch.float32)
    
    # Downloading and loading a dataset from the hub. data will be saved to ~/.cache/huggingface/datasets by default
    if dataset_name:
        dataset = load_dataset(dataset_name)
    else:
        dataset = load_dataset(
            "imagefolder"
            , data_dir = train_data_dir
        )
    
    train_data = dataset["train"]
    dataset["train"] = train_data.select(range(top_rows))
    print(dataset["train"])

    # Preprocessing the datasets. We need to tokenize inputs and targets.
    dataset_columns = list(dataset["train"].features.keys())
    image_column, caption_column = dataset_columns[0],dataset_columns[1]

    def tokenize_captions(examples, is_train=True):
        '''Preprocessing the datasets.We need to tokenize input captions and transform the images.'''
        captions = []
        for caption in examples[caption_column]:
            if isinstance(caption, str):
                captions.append(caption)
        inputs = tokenizer(
            captions, max_length=tokenizer.model_max_length, padding="max_length", truncation=True, return_tensors="pt"
        )
        return inputs.input_ids

    # Preprocessing the datasets.
    train_transforms = transforms.Compose(
        [
            transforms.Resize(
                resolution
                , interpolation=transforms.InterpolationMode.BILINEAR
            )
            , transforms.CenterCrop(resolution) if center_crop else transforms.RandomCrop(resolution)
            , transforms.RandomHorizontalFlip() if random_flip else transforms.Lambda(lambda x: x)
            , transforms.ToTensor()
            , transforms.Normalize([0.5], [0.5]) # [0,1] -> [-1,1]
        ]
    )

    def preprocess_train(examples):
        '''prepare the train data'''
        images                      = [image.convert("RGB") for image in examples[image_column]]
        examples["pixel_values"]    = [train_transforms(image) for image in images]
        examples["input_ids"]       = tokenize_captions(examples)
        return examples

    # only do this in the main process
    with accelerator.main_process_first():
        # Set the training transforms
        train_dataset = dataset["train"].with_transform(preprocess_train)
    
    def collate_fn(examples):
        pixel_values    = torch.stack([example["pixel_values"] for example in examples])
        pixel_values    = pixel_values.to(memory_format = torch.contiguous_format).float()
        input_ids       = torch.stack([example["input_ids"] for example in examples])
        return {"pixel_values": pixel_values, "input_ids": input_ids}
    
    # DataLoaders creation:
    train_dataloader = torch.utils.data.DataLoader(
        train_dataset
        , shuffle       = True
        , collate_fn    = collate_fn
        , batch_size    = train_batch_size
        , num_workers   = 0
    )

    print("Data Size:",len(train_dataloader))

    #num_update_steps_per_epoch  = math.ceil(len(train_dataloader) / gradient_accumulation_steps)
    num_update_steps_per_epoch  = math.ceil(len(train_dataloader) / train_batch_size)
    max_train_steps             = num_train_epochs * num_update_steps_per_epoch

    # initialize optimizer
    lora_layers = filter(lambda p: p.requires_grad, unet.parameters())
    optimizer = torch.optim.AdamW(
        lora_layers
        , lr            = learning_rate
        , betas         = (adam_beta1, adam_beta2)
        , weight_decay  = adam_weight_decay
        , eps           = adam_epsilon
    )

    # learn rate scheduler from diffusers's get_scheduler
    lr_scheduler = get_scheduler(
        lr_scheduler_name
        , optimizer             = optimizer
    )

    # Prepare everything with our `accelerator`.
    unet, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        unet, optimizer, train_dataloader, lr_scheduler
    )

    # set step count and progress bar
    max_train_steps = num_train_epochs*len(train_dataloader)
    progress_bar = tqdm(
        range(0, max_train_steps)
        , initial   = 0
        , desc      = "Steps"
        # Only show the progress bar once on each machine.
        , disable   = not accelerator.is_local_main_process,
    )

    # start train
    for epoch in range(num_train_epochs):
        unet.train()
        train_loss = 0.0
        for step, batch in enumerate(train_dataloader):
            # step 1. Convert images to latent space
            latents = vae.encode(batch["pixel_values"].to(dtype=weight_dtype)).latent_dist.sample()
            latents = latents * vae.config.scaling_factor

            # step 2. Sample noise that we'll add to the latents, latents provide the shape info. 
            noise = torch.randn_like(latents)

            # step 3. Sample a random timestep for each image
            batch_size = latents.shape[0]
            timesteps = torch.randint(
                low         = 0
                , high      = noise_scheduler.config.num_train_timesteps
                , size      = (batch_size,)
                , device    = latents.device
            )
            timesteps = timesteps.long()

            # step 4. Get the text embedding for conditioning
            encoder_hidden_states = text_encoder(batch["input_ids"])[0]

            # step 5. Add noise to the latents according to the noise magnitude at each timestep
            # (this is the forward diffusion process), provide to unet to get the prediction result
            noisy_latents = noise_scheduler.add_noise(latents, noise, timesteps)

            # step 6. Get the target for loss depend on the prediction type
            if noise_scheduler.config.prediction_type == "epsilon":
                target = noise
            elif noise_scheduler.config.prediction_type == "v_prediction":
                target = noise_scheduler.get_velocity(latents, noise, timesteps)
            else:
                raise ValueError(f"Unknown prediction type {noise_scheduler.config.prediction_type}")

            # step 7. Predict the noise residual and compute loss
            model_pred = unet(noisy_latents, timesteps, encoder_hidden_states).sample

            # step 8. Calculate loss
            loss = F.mse_loss(model_pred.float(), target.float(), reduction="mean")

            # step 9. Gather the losses across all processes for logging (if we use distributed training).
            avg_loss = accelerator.gather(loss.repeat(train_batch_size)).mean()
            train_loss += avg_loss.item() / gradient_accumulation_steps

            # step 10. Backpropagate
            accelerator.backward(loss)
            if accelerator.sync_gradients:
                params_to_clip = lora_layers
                accelerator.clip_grad_norm_(params_to_clip, max_grad_norm)
            
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

            # step 11. check optimization step and update progress bar
            if accelerator.sync_gradients:
                progress_bar.update(1)
                train_loss = 0.0
            
            logs = {"epoch": epoch,"step_loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0]}
            progress_bar.set_postfix(**logs)

    # Save the lora layers
    accelerator.wait_for_everyone()
    if accelerator.is_main_process:
        unet = unet.to(torch.float32)

        unwrapped_unet = accelerator.unwrap_model(unet)
        unet_lora_state_dict = convert_state_dict_to_diffusers(get_peft_model_state_dict(unwrapped_unet))

        weight_name = f"lora_{pretrained_model_name_or_path.split('/')[-1]}_rank{lora_rank}_s{max_train_steps}_r{resolution}_{diffusion_scheduler.__name__}_{formatted_date}.safetensors"   
        StableDiffusionPipeline.save_lora_weights(
            save_directory          = output_dir
            , unet_lora_layers      = unet_lora_state_dict
            , safe_serialization    = True
            , weight_name           = weight_name
        )

    accelerator.end_training()

if __name__ == "__main__":
    main()

2024-04-16 04:42:40.155567: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 04:42:40.155677: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 04:42:40.401519: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Configuration already exists at /root/.cache/huggingface/accelerate/default_config.yaml, will not override. Run `accelerate config` manually or pass a different `save_location`.


ValueError: AcceleratorState has already been initialized and cannot be changed, restart your runtime completely and pass `mixed_precision='fp16'` to `Accelerator()`.

In [22]:
#%%
from diffusers import StableDiffusionPipeline
import torch
from diffusers.utils import make_image_grid
from diffusers import EulerDiscreteScheduler

lora_name = "lora_stable-diffusion-v1-5_rank4_s200_r768_DDPMScheduler_20240214-122106.safetensors"
lora_model_path = f"./output_dir/{lora_name}"

device = "cuda:0"
pipe = StableDiffusionPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5"
    , torch_dtype = torch.bfloat16
).to(device)

pipe.load_lora_weights(
    pretrained_model_name_or_path_or_dict=lora_model_path
    , adapter_name = "az_lora"
)

prompt = "a toy bike. macro photo. 3d game asset"
nagtive_prompt = "low quality, blur, watermark, words, name"

pipe.set_adapters(
    ["az_lora"]
    , adapter_weights = [1.0]
)

pipe.scheduler = EulerDiscreteScheduler.from_config(pipe.scheduler.config)

images = pipe(
    prompt                      = prompt
    , nagtive_prompt            = nagtive_prompt
    , num_images_per_prompt     = 4
    , generator                 = torch.Generator(device).manual_seed(12)
    , width                     = 768
    , height                    = 768
    , guidance_scale            = 8.5
).images

pipe.to("cpu")
torch.cuda.empty_cache()
make_image_grid(images, cols = 2, rows = 2)

model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler/scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

text_encoder/model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

vae/config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

HFValidationError: Repo id must be in the form 'repo_name' or 'namespace/repo_name': './output_dir/lora_stable-diffusion-v1-5_rank4_s200_r768_DDPMScheduler_20240214-122106.safetensors'. Use `repo_type` argument if needed.